# Морфологический разбор старославянского языка при помощи DistilBERT

Задача: сделать pos-теггер для старославянского.

Данные: старославянский treebank от Universal Dependencies.

Наши данные находятся в формате CONLLU:

In [ ]:
with open('/content/drive/MyDrive/ML_training_data/chu_all.conllu', 'r') as f:
  chu_lines = f.readlines()

In [ ]:
for line in chu_lines[:10]:
  print(line)

# source = Codex Marianus, Matthew 9

# text = ꙇ вьлѣзъ въ корабь и҃съ прѣѣде ꙇ приде въ свои градъ

# sent_id = 38541

1	ꙇ	и	CCONJ	C-	_	6	cc	_	ref=MATT_9.1

2	вьлѣзъ	вълѣсти	VERB	V-	Case=Nom|Gender=Masc|Number=Sing|Strength=Strong|Tense=Past|VerbForm=Part|Voice=Act	6	advcl	_	ref=MATT_9.1

3	въ	въ	ADP	R-	_	4	case	_	ref=MATT_9.1

4	корабь	корабль	NOUN	Nb	Case=Acc|Gender=Masc|Number=Sing	2	obl	_	ref=MATT_9.1

5	и҃съ	исоусъ	PROPN	Ne	Case=Nom|Gender=Masc|Number=Sing	6	nsubj	_	ref=MATT_9.1

6	прѣѣде	прѣꙗхати	VERB	V-	Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	ref=MATT_9.1

7	ꙇ	и	CCONJ	C-	_	6	cc	_	ref=MATT_9.1



Поэтому для начала надо их предобработать и достать из них только слова и частеречные теги.

In [ ]:
import re

In [ ]:
def read_corp(file_path):

  with open(file_path, 'r') as f:
    raw_text = f.read()

  raw_docs = re.split(r'\n\t?\n', raw_text)
  token_docs = []
  tag_docs = []
  for doc in raw_docs:
      tokens = []
      tags = []
      for line in doc.split('\n'):
        if line:
          if line.split()[0] != '#':
            info = line.split('\t')
            tokens.append(info[1])
            tags.append(info[3])
      token_docs.append(tokens)
      tag_docs.append(tags)

  return token_docs, tag_docs

texts, tags = read_corp('/content/drive/MyDrive/ML_training_data/chu_all.conllu')

In [ ]:
len(texts)

6339

In [ ]:
texts[0]

['ꙇ',
 'вьлѣзъ',
 'въ',
 'корабь',
 'и҃съ',
 'прѣѣде',
 'ꙇ',
 'приде',
 'въ',
 'свои',
 'градъ']

In [ ]:
tags[0]

['CCONJ',
 'VERB',
 'ADP',
 'NOUN',
 'PROPN',
 'VERB',
 'CCONJ',
 'VERB',
 'ADP',
 'ADJ',
 'NOUN']

In [ ]:
train_texts = texts[:5071]
train_tags = tags[:5071]
val_texts = texts[5071:]
val_tags = tags[5071:]

Сделаем словарь всех уникальных тегов и каждому из них присвоим номер, чтобы можно было легко кодировать и декодировать теги:

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
tag2id

{'INTJ': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'PROPN': 4,
 'AUX': 5,
 'SCONJ': 6,
 'DET': 7,
 'VERB': 8,
 'CCONJ': 9,
 'X': 10,
 'PRON': 11,
 'NOUN': 12,
 'NUM': 13}

In [ ]:
! pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.6 MB/s eta 0:00:00


In [ ]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00


Мы будем работать с DistilBERT - более легковесной версией BERT. Также мы постараемся побольше использовать функционал библиотеки transformers.

In [ ]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 4

In [ ]:
# AutoTokenizer сам определяет, какой токенайзер взять
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Кодируем теги частей речи перед подачей в модель:

In [ ]:
encoded_tags_train = [[tag2id[tag] for tag in doc] for doc in train_tags]

In [ ]:
encoded_tags_val = [[tag2id[tag] for tag in doc] for doc in val_tags]

Нам нужно, чтобы количество лейблов соответствовало количеству токенов в закодированном предложении (т.е. с уже добавленным паддингом).

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(texts, tags):

    tokenized_inputs = tokenizer(texts,
                                 truncation=True, # обрезка слишком длинных последовательностей
                                 padding=True,
                                 is_split_into_words=True # предупреждаем, что вход поступит в виде списков токенов
                                 )

    labels = []
    for i, label in enumerate(tags):

        # достаем 1 текст
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        # идем по всем словам
        for word_idx in word_ids:

            # Некоторые специальные токены имеют id None. Мы даем им лейбл -100, чтобы модель их игнорировала
            if word_idx is None:
                label_ids.append(-100)

            # Логично, что если слово разделилось на subword-токены, их лейблы в пределах слова должны быть одинаковыми.
            # Если мы перешли на новое слово, добавляем его лейбл в список
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])

            # Всем следующим частям одного и того же слова мы даем или тот же лейбл, или -100, если label_all_tokens=False
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    return tokenized_inputs, labels

In [ ]:
tokenized_train, train_labels = tokenize_and_align_labels(train_texts, encoded_tags_train)

In [ ]:
print(train_labels[10])

[-100, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [ ]:
print(tokenized_train[10].tokens)

['[CLS]', 'в', '##ъ', '##с', '##т', '##а', '##в', '##ъ', 'х', '##о', '##д', '##и', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [ ]:
tokenized_val, val_labels = tokenize_and_align_labels(val_texts, encoded_tags_val)

In [ ]:
import torch

Как и pytorch, transformers может оперировать сущностями класса Dataset. Напишем такой класс, который не будет ничего делать, кроме хранения токенизированных текстов и их лейблов и подачи их в модель.

In [ ]:
class CHUDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
      # этот метод вызывается моделью, когда она учится
      # он определяет, в каком виде данные подаются в модель
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = CHUDataset(tokenized_train, train_labels)
val_dataset = CHUDataset(tokenized_val, val_labels)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(unique_tags))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Сначала объявим аргументы, которые будет использовать модель при тренировке:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"//content/sample_data/{model_name}-finetuned-1-pos-chu",
    overwrite_output_dir=True, # записываем каждый раз в один и тот же файл
    eval_strategy = "epoch", # оцениваем каждую эпоху
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size, # размер тренировочного батча на каждый процессор
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01, # регуляризация функции потерь
    push_to_hub=False, # не публиковать на Huggingface
    report_to="none" # disable wandb
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Коллаторы данных - это технические классы, формирующие батчи для подачи в модель:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
import evaluate

In [ ]:
metric = evaluate.load("seqeval")

Мы будем использовать метрику для оценки NER-моделей, которая возвращает precision, recall и f1-score для каждой сущности (в нашем случае - для каждого тега) и средние значения.

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Удаляем индексы специальных токенов
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Тренировка модели в transformers удобно осуществляется при помощи класса Trainer.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-37-b3676b2e1ff8>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.733100,0.563306,0.659880,0.604465,0.630958,0.818511
2,0.461700,0.470195,0.695811,0.645627,0.669780,0.854351
3,0.385000,0.438615,0.720730,0.678151,0.698793,0.868878
4,0.331800,0.429882,0.724923,0.687890,0.705921,0.874702
5,0.303800,0.428941,0.729442,0.693587,0.711063,0.876684


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CCONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:17

TrainOutput(global_step=6340, training_loss=0.484535112591572, metrics={'train_runtime': 1313.9577, 'train_samples_per_second': 19.297, 'train_steps_per_second': 4.825, 'total_flos': 2963965002798360.0, 'train_loss': 0.484535112591572, 'epoch': 5.0})

Оценка модели на валидационных данных:

In [ ]:
predictions, labels, _ = trainer.predict(val_dataset)
predictions = np.argmax(predictions, axis=2)

# Уберем игнорируемые токены и декодируем предсказанные токены
ids_predictions = [
    [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=ids_predictions, references=true_labels)
results

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CCONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:17

{'CONJ': {'precision': 0.7496740547588006,
  'recall': 0.7540983606557377,
  'f1': 0.7518796992481204,
  'number': 1525},
 'DJ': {'precision': 0.8047752808988764,
  'recall': 0.5690168818272096,
  'f1': 0.6666666666666667,
  'number': 1007},
 'DP': {'precision': 0.9809296781883194,
  'recall': 0.9558652729384437,
  'f1': 0.9682352941176471,
  'number': 861},
 'DV': {'precision': 0.8715909090909091,
  'recall': 0.7382098171318575,
  'f1': 0.7993746743095361,
  'number': 1039},
 'ERB': {'precision': 0.6045135035146134,
  'recall': 0.7049180327868853,
  'f1': 0.650866361282613,
  'number': 2318},
 'ET': {'precision': 0.5545454545454546,
  'recall': 0.4206896551724138,
  'f1': 0.47843137254901963,
  'number': 145},
 'NTJ': {'precision': 0.9152542372881356,
  'recall': 0.9,
  'f1': 0.9075630252100839,
  'number': 60},
 'OUN': {'precision': 0.6608587429993777,
  'recall': 0.6236054022313564,
  'f1': 0.6416918429003021,
  'number': 1703},
 'RON': {'precision': 0.7208774583963692,
  'recall': 

Попробуем загрузить и использовать модель с чекпойнта:

In [ ]:
from transformers import TokenClassificationPipeline

In [ ]:
# здесь можно поставить путь к файлу чекпойнта или адрес модели на huggingface
#checkpoint = 'annadmitrieva/old-church-slavonic-pos'
checkpoint = '/content/sample_data/distilbert-base-uncased-finetuned-1-pos-chu/checkpoint-3000'

In [ ]:
tokenizer_chu = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
model_chu = AutoModelForTokenClassification.from_pretrained(checkpoint)

In [ ]:
chubert = TokenClassificationPipeline(model=model_chu, tokenizer=tokenizer_chu, task="pos")

Device set to use cuda:0


In [ ]:
def postprocess_output(output):

  string = ''
  tags = []
  last_token_end = 0

  for token in output:
    # сначала обработаем pos-тег
    if '_' in token['entity']:
      pos = id2tag[int(token['entity'].split('_')[1])]
    else:
      pos = token['entity']

    if token['word'][0] != '#':
      # если токен - или начало слова, или полное слово
      if last_token_end != token['start']:
        string += ' '

      string += token['word']
      tags.append(pos)

    else:
      # если токен - середина слова или конец
      string += token['word'].replace('#', '')

    last_token_end = token['end']

  return list(zip(string.split(), tags))

In [ ]:
chubert('нъ да оувѣсте ѣко власть иматъ с҃нъ ч҃лвчскꙑ на земи отъпоущати грѣхꙑ')

[{'entity': 'LABEL_11',
  'score': 0.97578907,
  'index': 1,
  'word': 'н',
  'start': 0,
  'end': 1},
 {'entity': 'LABEL_11',
  'score': 0.97066474,
  'index': 2,
  'word': '##ъ',
  'start': 1,
  'end': 2},
 {'entity': 'LABEL_9',
  'score': 0.9484791,
  'index': 3,
  'word': 'д',
  'start': 3,
  'end': 4},
 {'entity': 'LABEL_9',
  'score': 0.9548244,
  'index': 4,
  'word': '##а',
  'start': 4,
  'end': 5},
 {'entity': 'LABEL_8',
  'score': 0.56610656,
  'index': 5,
  'word': 'оувѣсте',
  'start': 6,
  'end': 13},
 {'entity': 'LABEL_2',
  'score': 0.29271048,
  'index': 6,
  'word': 'ѣко',
  'start': 14,
  'end': 17},
 {'entity': 'LABEL_12',
  'score': 0.99301505,
  'index': 7,
  'word': 'в',
  'start': 18,
  'end': 19},
 {'entity': 'LABEL_12',
  'score': 0.99388266,
  'index': 8,
  'word': '##л',
  'start': 19,
  'end': 20},
 {'entity': 'LABEL_12',
  'score': 0.99360716,
  'index': 9,
  'word': '##а',
  'start': 20,
  'end': 21},
 {'entity': 'LABEL_12',
  'score': 0.9935407,
  'index

In [ ]:
postprocess_output(chubert('нъ да оувѣсте ѣко власть иматъ с҃нъ ч҃лвчскꙑ на земи отъпоущати грѣхꙑ'))

[('нъ', 'CCONJ'),
 ('да', 'SCONJ'),
 ('оувѣсте', 'VERB'),
 ('ѣко', 'PRON'),
 ('власть', 'NOUN'),
 ('иматъ', 'VERB'),
 ('снъ', 'NOUN'),
 ('ч҃лвчскꙑ', 'ADJ'),
 ('на', 'ADP'),
 ('земи', 'NOUN'),
 ('отъпоущати', 'VERB'),
 ('грѣхꙑ', 'PRON')]